The Newtonian Gravitational Force depends upon the masses of the two objects and the square of the distances between them. When one astronomical body acts upon another with force F12, the second body acts upon the first with equal and opposite force F21=-F12. Gravitational Force follows an inverse square law with regard to distance scaling that is exact in Newtonian gravity.

In [112]:
import numpy as np
import matplotlib.pyplot as plt

def NewtonianForce(mass1,mass2, r):
    return (6.67408*10**-11)*mass1*mass2/r/r;

To compute an orbit, it is necessary to solve a second order differential equation. 

F=ma

For a two body problem:

F12x=m1 d^2 x1/dt^2

This can be decomposed into two coupled differential equations.

F12x= m1 d v1/dt

v1= d x1/dt

There is also a y component and a z component. 
It is also necessary to track the position of the second particle.

Recall that once a force F12 is split into its components in the x and y direction, F12 will no longer exactly be given by the NewtonainForce equation given above but rather the magnitude of F12x, F12y, and F12z combined will equal that total. The nagnitude of a vector is found by squaring and summing the components then taking the square roots. So, sqrt(F12x^2+F12y^2+F12z^2)=FNewtonianForce

Assume, for the purposes of a two body problem, the orbit is confined to the x-y plane. 

Define polar coordinates phi in the x-y plane beginning from the x axis and proceeding toward the y axis. Then F12x=F12 cos phi and F12y= F12 sin phi.

F12x=x/sqrt(x^2+y^2)*F12

F12y = y/sqrt(x^2+y^2)*F12

# Numerical solutions to differential equations

To integrate a differential equation numerically, simply differentiate both of the two coupled equations, one time steop at a time, numerically. But how is this done numerically? There are a couple of ways. First consider how a derivative is defined. A derivative is a limit of a difference divided by a timestep. The euler derivative makes this definition discrete for small time steps. 

In [113]:
def euler(h,t, x,y,z,f,debugprint):
    if(debugprint):
        print(h,t,x)
    step2=f(t+h,x+h,y,z)
    step1=f(t,h,y,z)
    if(debugprint):
        print("step1", step1)
        print("step2", step2)
    rise=(step2-step1)
    run= 2.*h
    slope=rise/run
    if(debugprint):
        print("rise", rise)
        print("run", run)
        print("slope",slope)
    print(slope)
    return slope

The euler routine essentially just finds the slope in a small region of the graph. 

The Runga Kutta numerical derivative is a fourth order approximation to the derivative that accounts for the curvature and is better for something like an orbit that fundamentally has curvature to it where orvershoot near a minima may matter. 

In [114]:
def RK4(h,t,x,y,z,f): #not a finite difference so no step in y
    print(t,x,y,z,h)
    k1= h*f(t,x,y,z)
    print("k1",k1)
    k2=h*f(t+h/2.,x+k1/2.,y,z)
    print("k2",k2)
    k3=h*f(t+h/2.,x+k2/2.,y,z)
    print("k3",k3)
    k4=h*f(t+h,x+k3,y,z)
    print("k4",k4)
    return t+h, x+1/6.*(k1+2.*k2+2.*k3+k4)



In [115]:
def RK4nodebug(h,t,x,y,z,f): #not a finite difference so no step in y
    k1= h*f(t,x,y,z)
    #print("k1",k1)
    k2=h*f(t+h/2.,x+k1/2.,y,z)
    #print("k2",k2)
    k3=h*f(t+h/2.,x+k2/2.,y,z)
    #print("k3",k3)
    k4=h*f(t+h,x+k3,y,z)
    #print("k4",k4)
    return t+h, x+1/6.*(k1+2.*k2+2.*k3+k4)



# Verifying the RK4 routine numerically

First I verify the RK4 by examining the behavior for a differential equation where the right hand side is a fourth order polynomial. This should have a truncation error that is on the order of the fourth order polynomial itself

In [116]:
def polynomial4(t,x,a,b,c,d,e):
    p=a*t**4+b*t**3+c*t**2+d*t+e
    #print(pi)
    return p
    

def polymaker(t,x,y,z,a,b,c,d,e):
    def poly(t,x,y,z):
        return polynomial4(t,x,a,b,c,d,e)
    return poly
        
        

In [117]:
import numpy as np
t=0.
x=0.1
y=0.1
z=0.1
print(t,x)
polyfn=polymaker(t,x,y,z,1.,0.,0.,0.,0.)

0.0 0.1


In [118]:
#import matplotlib.pyplot as plt
#plt.plot(t,polyfn(t,x))

In [119]:
outp=polyfn(t,x,y,z) #this is correct to order of magnitude
print(outp)

0.0


In [120]:
rk4polyout=RK4(.01,t,x,y,z,polyfn)
print(x)
print(outp+x)
print(rk4polyout)

0.0 0.1 0.1 0.1 0.01
k1 0.0
k2 6.25e-12
k3 6.25e-12
k4 1e-10
0.1
0.1
(0.01, 0.10000000002083334)


In [121]:
dt=0.01
numsteps=2
intvals=[]
tvals=[]
for i in np.arange(1,numsteps):
    t=0.+i*dt
    tnew,valueofintegral=RK4(dt,t,x,y,z,polyfn)
    #print(valueofintegral)
    intvals.append(valueofintegral)
    tvals.append(t)
print(intvals)
print(tvals)

0.01 0.1 0.1 0.1 0.01
k1 1e-10
k2 5.062499999999999e-10
k3 5.062499999999999e-10
k4 1.6e-09
[0.10000000062083333]
[0.01]


k1 1e-10
k2 5.062499999999999e-10
k3 5.062499999999999e-10
k4 1.6e-09
[6.208333333333332e-10]
[0.01]

In [122]:
2* 5.062499999999999e-10+ 2* 5.062499999999999e-10 

2.0249999999999995e-09

In [123]:
2.0249999999999995e-09+1.6e-09+1e-10

3.7249999999999994e-09

In [124]:
polyfn(0.1,0.1,0.1,0.1)

0.00010000000000000002

In [125]:
.01*polyfn(0.01,0.1,0.1,0.1)

1e-10

In [126]:
3.7249999999999994e-09 /6.

6.208333333333332e-10

# 1/6.*(k1+2.*k2+2.*k3+k4)

The value returned is not analyitically correct for the first step but I don't see the error yet.  

In [127]:
dt=0.01
numsteps=30
integralvals=[]
tvals=[]
for i in np.arange(1,numsteps):
    t=0.+i*dt
    tnew,valueofintegral=RK4nodebug(dt,t,x,y,z,polyfn)
    #print(valueofintegral)
    integralvals.append(valueofintegral)
    tvals.append(t)
print(np.array(integralvals)-.1)
print(tvals)
print(len(integralvals),len(tvals))

[6.20833329e-10 4.22083334e-09 1.56208333e-08 4.20208333e-08
 9.30208333e-08 1.80620833e-07 3.19220833e-07 5.25620833e-07
 8.19020833e-07 1.22102083e-06 1.75562083e-06 2.44922083e-06
 3.33062083e-06 4.43102083e-06 5.78402083e-06 7.42562083e-06
 9.39422083e-06 1.17306208e-05 1.44780208e-05 1.76820208e-05
 2.13906208e-05 2.56542208e-05 3.05256208e-05 3.60600208e-05
 4.23150208e-05 4.93506208e-05 5.72292208e-05 6.60156208e-05
 7.57770208e-05]
[0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29]
29 29


The analyitic solution to the differential equation

dx/dt = t^4 

is 

x= 1/5 * t^5

Below the output is shown. The results are plotted for scaling comparison. It doesn't look like it scales correctly. '

In [128]:
def ODEsolnanalytic(t,t0,x0):
    return 0.2*(t)**5.+x0

In [129]:
solnanalytic=[]
for i in np.arange(len(tvals)):
    solnanalytic.append(ODEsolnanalytic(tvals[i],tvals[0],0.1))
print(np.array(solnanalytic)-.1)
print(len(solnanalytic))

[2.00000017e-11 6.39999997e-10 4.86000000e-09 2.04800000e-08
 6.25000000e-08 1.55520000e-07 3.36140000e-07 6.55360000e-07
 1.18098000e-06 2.00000000e-06 3.22102000e-06 4.97664000e-06
 7.42586000e-06 1.07564800e-05 1.51875000e-05 2.09715200e-05
 2.83971400e-05 3.77913600e-05 4.95219800e-05 6.40000000e-05
 8.16820200e-05 1.03072640e-04 1.28726860e-04 1.59252480e-04
 1.95312500e-04 2.37627520e-04 2.86978140e-04 3.44207360e-04
 4.10222980e-04]
29


In [130]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(tvals,integralvals, size=15, line_color="blue", fill_color="blue", fill_alpha=0.5)
p.circle(tvals,solnanalytic, size=15, line_color="green",fill_color="green", fill_alpha=0.5)
#p.circle(tvals,np.array(solnanalytic)-np.array(integralvals),size=15, line_color="red", fill_color="red", fill_alpha=0.5)

show(p) # show the results

Loading BokehJS ...

The truncation error is on the order of the fifth order analytic solution, not the fourth order polynomial right hand side of the Ordinary Differential Equation.

In [131]:
print(rk4polyout) 
#There are multiple items in the y output, which is confusing. trace this over time and plot an evolution and compare to exact solution, which I have analytically written down. steve points out truncation error
print(((rk4polyout[1]-x-outp)/x))
#order4 polynomial error (comparable in size to polyfn itself, relative error with it is one)

(0.01, 0.10000000002083334)
2.0833335057091062e-10


# Try using euler funciton to compare

In [132]:
dt=0.01
numsteps=2
eintegralvals=[]
tvals=[]
for i in np.arange(1,numsteps):
    t=0.+i*dt
    valueofintegral=euler(dt,t,x,y,z,polyfn,True)
    #print(valueofintegral)
    eintegralvals.append(valueofintegral)
    tvals.append(t)
print(np.array(eintegralvals))
print(tvals)

0.01 0.01 0.1
step1 1e-08
step2 1.6e-07
rise 1.5e-07
run 0.02
slope 7.499999999999999e-06
7.499999999999999e-06
[7.5e-06]
[0.01]


Find slope. First find rise. Then divide by run (2h). 

step2-step1 

f(t+h,x+h)-f(t,x)

In [133]:
1.6e-07-1e-08

1.5e-07

In [134]:
1.5e-07/2/0.01

7.499999999999999e-06

In [135]:
dt=0.01
numsteps=30
eintegralvals=[]
tvals=[]
for i in np.arange(1,numsteps):
    t=0.+i*dt
    valueofintegral=eulernodebug(dt,t,x,y,z,polyfn)
    #print(valueofintegral)
    eintegralvals.append(valueofintegral)
    tvals.append(t)
print(np.array(eintegralvals))
print(tvals)
print(len(eintegralvals),len(tvals))

[7.50000e-06 3.25000e-05 8.75000e-05 1.84500e-04 3.35500e-04 5.52500e-04
 8.47500e-04 1.23250e-03 1.71950e-03 2.32050e-03 3.04750e-03 3.91250e-03
 4.92750e-03 6.10450e-03 7.45550e-03 8.99250e-03 1.07275e-02 1.26725e-02
 1.48395e-02 1.72405e-02 1.98875e-02 2.27925e-02 2.59675e-02 2.94245e-02
 3.31755e-02 3.72325e-02 4.16075e-02 4.63125e-02 5.13595e-02]
[0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29]
29 29


In [140]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(plot_width=400, plot_height=400, y_axis_type="log")

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(tvals,np.array(integralvals)-0.1, size=15, line_color="blue", fill_color="blue", fill_alpha=0.5)
p.circle(tvals,np.array(solnanalytic)-0.1, size=15, line_color="green",fill_color="green", fill_alpha=0.5)
#p.circle(tvals,np.array(solnanalytic)-np.array(integralvals),size=15, line_color="red", fill_color="red", fill_alpha=0.5)
p.circle(tvals,eintegralvals, size=15, line_color="red", fill_color="red", fill_alpha=0.5)
show(p) # show the results

Loading BokehJS ...

# Two Body, Earth and Sun

Generate iniitial data for two body problem, Sun and Earth, using reduced mass framework.

In [10]:
import math
def InitialData():
    random.seed(a=9001)
    
    #initially use an in plane orbit with random starting locations relative to the x axis
    phi=np.array([0,math.pi])
    orbitangle=np.zeros(2)
    #orbitalradius=np.random.uniform(.1,50,2)
    orbitalradius=np.ones(2)
    #start with circular orbits
    #eccentricity=np.random.uniform(0.,.1,2)
    eccentricity=np.zeros(2)
    #magnitude=np.random.uniform(-20,-30,2) #absolute not apparent maginutde
    #magsun=-26.832
    masssun=1.989*10**30
    lsun=3.828*10**26
    massearth=5.9722*10**24
    #luminosity=lsun*10**(0.4*(magnitude-magsun))
    #masses= invertIMF(luminosity,lsun,masssun) #Initial mass function for Main Sequence
    #masses=np.random.uniform(.7,5.) #replace with IMF
    masses=np.array([masssun,massearth])

    return phi,orbitangle,orbitalradius,eccentricity, masses



In [11]:
import random,numpy as np
initdat=InitialData()

In [12]:
print(initdat)

(array([0.        , 3.14159265]), array([0., 0.]), array([1., 1.]), array([0., 0.]), array([1.9890e+30, 5.9722e+24]))


Transform cylindrical initial data to x,y,z, velocity, acceleration initial data

In [13]:
def getxyuv(initdat):
    phi,orbitangle,orbitalradius,eccentricity, masses=initdat
    #print(orbitalradius, phi, np.cos(phi), np.sin(phi))
    metersperAU=149597870700
    Gconstant=6.408*10**-11
    x0=orbitalradius*np.cos(phi)*metersperAU
    y0=orbitalradius*np.sin(phi)*metersperAU
    z0=np.zeros(2)
    

    
    #start at perihelion for both (eliptical, doesn't generalize to three body)
    #actually start with circular orbit
    ux0=np.zeros(2)*149597870700
    #centrepital force balances gravitational force
    reducedmass=np.zeros(2)
    print(masses)
    for i in np.arange(2):
        j=(i+1)%2 #reverse masses
        reducedmass[i]=masses[i]*masses[j]/np.sum(masses)
    print(reducedmass)
    rphys=np.zeros(len(masses))
    for i in np.arange(2):
        rphys[i]=orbitalradius[i]*metersperAU*reducedmass[i]/masses[i]
    x0=rphys*np.cos(phi)
    y0=rphys*np.sin(phi)
    z0=np.zeros(2)
    print(rphys)
    F=(Gconstant*reducedmass**2/rphys**2)
    print(F)
    #centF=reducedmass*v**2/rphys
    #centF=accel
    v=np.sqrt(Gconstant*reducedmass/rphys)
    print(v)
    ux0=v*np.sin(phi)
    uy0=v*np.cos(phi) #initial data in y only
    #evolve in plane only
    #there is a units problem that needs to be fixed
    #velocity initial conditions are not trivial. 
    uz0=np.zeros(2)
    
    #circular orbit   #a=omega^2 * r #v=omega*r #omega=v/r
    omega=v/rphys
    print(omega)
    omegatrue=np.mean(omega) #shouldaccount for numerical effects
    print(omegatrue)
    a=omegatrue**2*rphys
    print("a",a)
    print(phi)
    ax0=-a*np.cos(phi)
    ay0=-a*np.sin(phi)
    az0=np.zeros(2)
    
    
    return reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0

In [14]:
xyuva=getxyuv(initdat)
print(xyuva)#In SI units
print(xyuva[1][0])
print(xyuva[0][0]/xyuva[0][1])
print(xyuva)

[1.9890e+30 5.9722e+24]
[5.97218207e+24 5.97218207e+24]
[4.49183369e+05 1.49597422e+11]
[1.13276871e+28 1.02126951e+17]
[29188.7795877     50.57847341]
[6.49818796e-02 3.38097227e-10]
0.032490939958166556
a [4.74185445e+02 1.57924190e+08]
[0.         3.14159265]
(array([5.97218207e+24, 5.97218207e+24]), array([ 4.49183369e+05, -1.49597422e+11]), array([0.00000000e+00, 1.83204003e-05]), array([0., 0.]), array([0.00000000e+00, 6.19407656e-15]), array([29188.7795877 ,   -50.57847341]), array([0., 0.]), array([-4.74185445e+02,  1.57924190e+08]), array([-0.00000000e+00, -1.93401354e-08]), array([0., 0.]))
449183.36891987134
1.0
(array([5.97218207e+24, 5.97218207e+24]), array([ 4.49183369e+05, -1.49597422e+11]), array([0.00000000e+00, 1.83204003e-05]), array([0., 0.]), array([0.00000000e+00, 6.19407656e-15]), array([29188.7795877 ,   -50.57847341]), array([0., 0.]), array([-4.74185445e+02,  1.57924190e+08]), array([-0.00000000e+00, -1.93401354e-08]), array([0., 0.]))


Time stepping routine using Euler method and Newtonian differential equation for two body system

In [15]:
def timestep(step,t,dt,reducedmass,xi,yi,zi, vxi, vyi, vzi, axi, ayi, azi):
    xii=np.zeros(np.size(xi))
    vxii=np.zeros(np.size(vx))
    yii=np.zeros(np.size(yi))
    vyii=np.zeros(np.size(vy))
    zii=np.zeros(np.size(vzi))
    vzii=np.zeros(np.size(vzi))
    rii=np.zeros(np.size(xi))
    axii=axi
    ayii=ayi
    azii=azi
    
    for m in np.arange(len(x)):
        #m represents choices of mass
        i=step
        
        xii[m] = xi[m] + dt*vxi[m]
        #print(xii)
        vxii[m] = vxi[m] + dt*axi[m]
        #print(vxii)
        yii[m]= yi[m] + dt*vyi[m]
        vyii[m] = vyi[m] + dt*ayi[m]
        zii[m]= zi[m] + dt*vzi[m]
        vzii[m] = vzi[m] + dt*azi[m]
        rii[m]=np.sqrt(xi[m]**2+yi[m]**2+zi[m]**2)
    
    
    Gconstant=6.408*10**-11
    for k in np.arange(len(rii)):
        for j in np.arange(len(rii)):
            if j!=k:
                rreljk=np.abs((xi[j] - xi[k])**2+(yi[j]-yi[k])**2+(zi[j]-zi[k])**2)**(1./2.)
                axii[j]+=Gconstant*reducedmass[k]*(xi[j]  - xi[k])/rreljk**3
                ayii[j]+=Gconstant*reducedmass[k]*(yi[j]  - yi[k])/rreljk**3
                azii[j]+=Gconstant*reducedmass[k]*(zi[j]  - zi[k])/rreljk**3
    #print(xii)
    return reducedmass, xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii
                    

Run over several time steps and print

In [16]:
dt=0.01*31556926 #seconds per year
numsteps=5
mass,x,y,z,vx,vy,vz,ax,ay,az=xyuva
for i in np.arange(1,numsteps):
    t=0.+i*numsteps*dt
    mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(i,t,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
    print(x,y,vx,vy,ax,ay)
    #print(ay) #forces should be equal and opposite, but in reduced mass framework accelerations are also equal and opposite
    #accelerations should evolve from y to x with time in a sinusoidal manner even in reduced mass framework
    #print(ax)
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(2,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(3,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(4,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 

[ 4.49183369e+05 -1.49597422e+11] [ 9.21108157e+09 -1.59610114e+07] [-1.49638350e+08  4.98360199e+13] [29188.7795877    -50.58457657] [-4.74185445e+02  1.57924190e+08] [-2.09418125e-24 -1.93401354e-08]
[-4.72212629e+13  1.57267158e+19] [ 1.84221631e+10 -3.19239488e+07] [-2.99276700e+08  9.96720398e+13] [29188.7795877    -50.59067972] [-4.74185445e+02  1.57924190e+08] [ 1.04874102e-09 -2.03888765e-08]
[-1.41663790e+14  4.71801476e+19] [ 2.76332447e+10 -4.78888122e+07] [-4.4891505e+08  1.4950806e+14] [29188.77991865   -50.59711382] [-4.74185445e+02  1.57924190e+08] [ 1.04874102e-09 -2.03888765e-08]
[-2.83327580e+14  9.43602954e+19] [ 3.68443264e+10 -6.38557059e+07] [-5.9855340e+08  1.9934408e+14] [29188.7802496    -50.60354792] [-4.74185445e+02  1.57924190e+08] [ 1.04874102e-09 -2.03888765e-08]


Object for storing the orbital differential equation and evolving using implicit RK4 using Newtonian differential equation for two body system

In [17]:
class OrbitDiffEq:
    def __init__(self,reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0):
        self.reducedmass=reducedmass
        self.xi=x0
        self.yi=y0
        self.zi=z0
        self.vxi=ux0
        self.vyi=uy0
        self.vzi=uz0
        self.axi=ax0
        self.ayi=ay0
        self.azi=az0
    def dxidt(self,t):
        return self.vxi
    def dyidt(self,t):
        return self.vyi
    def dzidt(self,t):
        return self.vzi
    def dvxidt(self,t):
        #return axi[m]
        axii=self.axi
        rii=np.sqrt(self.xi**2+self.yi**2+self.zi**2)
        Gconstant=6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(len(rii)):
                if j!=k:
                    rreljk=np.abs((self.xi[j] - self.xi[k])**2+(self.yi[j]-self.yi[k])**2+(self.zi[j]-self.zi[k])**2)**(1./2.)
                    axii[j]+=Gconstant*self.reducedmass[k]*(self.xi[j]  - self.xi[k])/rreljk**3
        self.axi=axii
        return axii
    def dvyidt(self,t):
        #return axi[m]
        ayii=self.ayi
        rii=np.sqrt(self.xi**2+self.yi**2+self.zi**2)
        Gconstant=6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(len(rii)):
                if j!=k:
                    rreljk=np.abs((self.xi[j] - self.xi[k])**2+(self.yi[j]-self.yi[k])**2+(self.zi[j]-self.zi[k])**2)**(1./2.)
                    ayii[j]+=Gconstant*self.reducedmass[k]*(self.yi[j]  - self.yi[k])/rreljk**3
        self.ayi=ayii
        return ayii
    def dvzidt(self,t):
        #return axi[m]
        azii=self.azi
        rii=np.sqrt(self.xi**2+self.yi**2+self.zi**2)
        Gconstant=6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(len(rii)):
                if j!=k:
                    rreljk=np.abs((self.xi[j] - self.xi[k])**2+(self.yi[j]-self.yi[k])**2+(self.zi[j]-self.zi[k])**2)**(1./2.)
                    azii[j]+=Gconstant*self.reducedmass[k]*(self.zi[j]  - self.zi[k])/rreljk**3
        self.azi=azii
        return azii
    def updateINTERNAL(self,xii,yii,zii,vxii,vyii,vzii):
        self.xi=xii
        self.yi=yii
        self.zi=zii
        self.vxi=vxii
        self.vyi=vyii
        self.vzi=vzii
        return self
    def update(self,xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii):
        self.xi=xii
        self.yi=yii
        self.zi=zii
        self.vxi=vxii
        self.vyi=vyii
        self.vzi=vzii
        self.axi=axii
        self.ayi=ayii
        self.azi=azii
    def print2D(self):
        print(self.xi,self.yi,self.vxi,self.vyi,self.axi,self.ayi)
        return self
    def list2D(self):
        return self.xi,self.yi,self.vxi,self.vyi,self.axi,self.ayi
    def timestepRK4ODE(self,step,t,dt):

    
        h=dt
        #tnew,ynew, intval=RK4(h,t,y,f)
        #m represents choices of mass
        i=step
        
        tnew,intvalx=RK4implicit(h,t,self.dxidt)
        xii = intvalx
        tnew,intvalvx=RK4implicit(h,t,self.dvxidt)
        vxii=intvalvx
        tnew,intvaly=RK4implicit(h,t,self.dyidt)
        yii = intvaly
        tnew,intvalvy=RK4implicit(h,t,self.dvyidt)
        vyii=intvalvy
        tnew,intvalz=RK4implicit(h,t,self.dzidt)
        zii = intvalz
        tnew,intvalvz=RK4implicit(h,t,self.dvzidt)
        vzii=intvalvz
 
        #print(xii)
        self.updateINTERNAL(xii,yii,zii,vxii,vyii,vzii)
        return reducedmass, xii,yii,zii,vxii,vyii,vzii,self.axi,self.ayi,self.azi

Implicit RK4

In [18]:
def RK4implicit(h,t,f): #not a finite difference so no step in y
    k1= h*f(t)
    k2=h*f(t+h/2.)
    k3=h*f(t+h/2.)
    k4=h*f(t+h)
    return t+h, f(t)+1/6.*(k1+2.*k2+2.*k3+k4)




Initialize iniitial conditions 

In [19]:
reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0=xyuva
ODE= OrbitDiffEq(reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0)
ODE.print2D()

[ 4.49183369e+05 -1.49597422e+11] [0.00000000e+00 1.83204003e-05] [0.00000000e+00 6.19407656e-15] [29188.7795877    -50.57847341] [-4.74185445e+02  1.57924190e+08] [ 1.04874102e-09 -2.03888765e-08]


Extract coordinates

In [20]:
mass,x,y,z,vx,vy,vz,ax,ay,az=xyuva
print(x,y,vx,vy,ax,ay)
print(xyuva)

[ 4.49183369e+05 -1.49597422e+11] [0.00000000e+00 1.83204003e-05] [0.00000000e+00 6.19407656e-15] [29188.7795877    -50.57847341] [-4.74185445e+02  1.57924190e+08] [ 1.04874102e-09 -2.03888765e-08]
(array([5.97218207e+24, 5.97218207e+24]), array([ 4.49183369e+05, -1.49597422e+11]), array([0.00000000e+00, 1.83204003e-05]), array([0., 0.]), array([0.00000000e+00, 6.19407656e-15]), array([29188.7795877 ,   -50.57847341]), array([0., 0.]), array([-4.74185445e+02,  1.57924190e+08]), array([ 1.04874102e-09, -2.03888765e-08]), array([0., 0.]))


Evolve the differential equation and print

In [21]:
dt=0.001*31556926 #seconds per year
numsteps=5
reducedmass,x,y,z,vx,vy,vz,ax,ay,az=xyuva
ODE.print2D()
for i in np.arange(1,numsteps):
    t=0.+i*numsteps*dt
    reducedmass, x,y,z,vx,vy,vz,ax,ay,az=ODE.timestepRK4ODE(i,t,dt)
    ODE.print2D()
    ODE.update(x,y,z,vx,vy,vz,ax,ay,az)



[ 4.49183369e+05 -1.49597422e+11] [0.00000000e+00 1.83204003e-05] [0.00000000e+00 6.19407656e-15] [29188.7795877    -50.57847341] [-4.74185445e+02  1.57924190e+08] [ 1.04874102e-09 -2.03888765e-08]
[0.0000000e+00 1.9547221e-10] [ 9.21137346e+08 -1.59615172e+06] [-1.49643092e+07  4.98375992e+12] [ 3.30960916e-05 -6.43430654e-04] [-4.74185445e+02  1.57924190e+08] [ 1.04874102e-09 -2.03888765e-08]
[-4.72242562e+11  1.57277127e+17] [  1.04444401 -20.30533698] [-1.49643092e+07  4.98375992e+12] [ 35.46219366 -35.46280399] [-4.74185445e+02  1.57924190e+08] [ 0.00224736 -0.00224738]
[-4.72242562e+11  1.57277127e+17] [ 1119113.28317089 -1119132.54406386] [-1.49643092e+07  4.98375992e+12] [ 70.92210685 -70.92271719] [-4.74185445e+02  1.57924190e+08] [ 0.00224736 -0.00224738]
[-4.72242562e+11  1.57277127e+17] [ 2238154.59982401 -2238173.86071698] [-1.49643092e+07  4.98375992e+12] [ 70.92210685 -70.92271719] [-4.74185445e+02  1.57924190e+08] [ 0.00224736 -0.00224738]


In [22]:
secperyr=31556926

Evolve the differential equation and store in a pandas data frame for later plotting

In [23]:
ODE2= OrbitDiffEq(reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0)



In [24]:
dt=0.01*31556926 #seconds per year
numsteps=5
reducedmass,x,y,z,vx,vy,vz,ax,ay,az=xyuva
datainit=[[0,0.0,x,y,z,vx,vy,vz,ax,ay,az]]
import pandas as pd
columns =['step','time','x' ,'y','z','vx','vy','vz','ax','ay','az']
df = pd.DataFrame(data=datainit, columns=columns)




In [25]:
display(df)

,step,time,x,y,z,vx,vy,vz,ax,ay,az
0,0,0.0,"[449183.36891987134, -149597421516.63107]","[0.0, 1.8320400342103965e-05]","[0.0, 0.0]","[0.0, 6.194076557160162e-15]","[29188.779587697383, -50.57847341349946]","[0.0, 0.0]","[-474.1854448655729, 157924190.42823714]","[0.002247362733938104, -0.0022473820740735355]","[0.0, 0.0]"


In [26]:
for i in np.arange(1,numsteps):
    t=0.+i*numsteps*dt
    reducedmass, x,y,z,vx,vy,vz,ax,ay,az=ODE2.timestepRK4ODE(i,t,dt)
    df.loc[i]=i,t,x,y,z,vx,vy,vz,ax,ay,az
    ODE2.update(x,y,z,vx,vy,vz,ax,ay,az)


In [27]:
display(df)

,step,time,x,y,z,vx,vy,vz,ax,ay,az
0,0,0.0,"[449183.36891987134, -149597421516.63107]","[0.0, 1.8320400342103965e-05]","[0.0, 0.0]","[0.0, 6.194076557160162e-15]","[29188.779587697383, -50.57847341349946]","[0.0, 0.0]","[-474.1854447800713, 157924190.428237]","[0.0022698376326943694, -0.002269856972829801]","[0.0, 0.0]"
1,1,1577846.3,"[0.0, 1.9546663496029367e-09]","[9211110763.572353, -15961062.00550111]","[0.0, 0.0]","[-149638824.11095357, 49836177833728.28]","[709.2008422631583, -709.2069454347248]","[0.0, 0.0]","[-474.1854447800713, 157924190.428237]","[0.0022698376326943694, -0.002269856972829801]","[0.0, 0.0]"
2,2,3155692.6,"[-47221562630787.875, 1.5726815596395868e+19]","[223802694.18520382, -223804620.16464195]","[0.0, 0.0]","[-149638824.0974627, 49836177833728.26]","[712.7470583226017, -712.7531614941684]","[0.0, 0.0]","[-474.1854447800713, 157924190.428237]","[0.0022698376326943694, -0.002269856972829801]","[0.0, 0.0]"
3,3,4733538.9,"[-47221562626530.57, 1.5726815596395866e+19]","[224921774.50909853, -224923700.4885367]","[0.0, 0.0]","[-149638824.0974627, 49836177833728.26]","[716.2932519071464, -716.2993550787132]","[0.0, 0.0]","[-474.1854447800713, 157924190.428237]","[0.0022698376326943694, -0.002269856972829801]","[0.0, 0.0]"
4,4,6311385.2,"[-47221562626530.57, 1.5726815596395866e+19]","[226040847.74058372, -226042773.72002184]","[0.0, 0.0]","[-149638824.0974627, 49836177833728.26]","[716.2932519071464, -716.2993550787132]","[0.0, 0.0]","[-474.1854447800713, 157924190.428237]","[0.0022698376326943694, -0.002269856972829801]","[0.0, 0.0]"


In [28]:
xser=df['x']
xser
xser[1][0]

0.0

Each pandas column has two entries because there are two bodies, the sun and the earth. These need to be split into columns one and two in a new table for each pandas column in the original table. The following routine automates extraction of these entries for each column. 

In [29]:
def mergeStarDataIntoDataFrame(col):
    rawdataser=df[col]
    numstars=len(rawdataser[0])
    columns=np.arange(numstars)
    numentries=len(rawdataser)
    index=np.arange(numentries)
    dfnew=pd.DataFrame(columns=columns,index=index)
    for coln in np.arange(numstars):
        datacollist=[]
        for row in np.arange(numentries):
            datacollist.append(rawdataser[row][coln])
        dfnew[coln]=datacollist
    return dfnew

In [30]:
dfx=mergeStarDataIntoDataFrame('x')

In [31]:
display(dfx)

,0,1
0,4.491834e+05,-1.495974e+11
1,0.000000e+00,1.954666e-09
2,-4.722156e+13,1.572682e+19
3,-4.722156e+13,1.572682e+19
4,-4.722156e+13,1.572682e+19


In [32]:
dfy=mergeStarDataIntoDataFrame('y')

In [33]:
display(dfy)

,0,1
0,0.000000e+00,1.832040e-05
1,9.211111e+09,-1.596106e+07
2,2.238027e+08,-2.238046e+08
3,2.249218e+08,-2.249237e+08
4,2.260408e+08,-2.260428e+08


In [34]:
dfvx=mergeStarDataIntoDataFrame('vx')

In [35]:
display(dfvx)

,0,1
0,0.000000e+00,6.194077e-15
1,-1.496388e+08,4.983618e+13
2,-1.496388e+08,4.983618e+13
3,-1.496388e+08,4.983618e+13
4,-1.496388e+08,4.983618e+13


In [36]:
dfvy=mergeStarDataIntoDataFrame('vy')

In [37]:
display(dfvy)

,0,1
0,29188.779588,-50.578473
1,709.200842,-709.206945
2,712.747058,-712.753161
3,716.293252,-716.299355
4,716.293252,-716.299355


In [38]:
dfax=mergeStarDataIntoDataFrame('ax')

In [39]:
display(dfax)

,0,1
0,-474.185445,1.579242e+08
1,-474.185445,1.579242e+08
2,-474.185445,1.579242e+08
3,-474.185445,1.579242e+08
4,-474.185445,1.579242e+08


In [40]:
dfay=mergeStarDataIntoDataFrame('ay')

In [41]:
display(dfay)

,0,1
0,0.00227,-0.00227
1,0.00227,-0.00227
2,0.00227,-0.00227
3,0.00227,-0.00227
4,0.00227,-0.00227


In [42]:
df['time'].tolist()

[0.0, 1577846.3, 3155692.6, 4733538.9, 6311385.2]

Plots time versus position of Earth, in reduced mass coordinates

In [43]:
from bokeh.plotting import figure, output_notebook, show
p=figure(plot_width=300,plot_height=300,x_axis_label="Time (s)",y_axis_label="y displacement (m), Earth")
show(p)
p.circle(x=np.array(df['time'].tolist()),y=np.array(dfy[1].tolist()))
show(p)
output_notebook()

W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='a57969ce-35b1-4953-bcd5-a4c19dd14dc2', ...)


Loading BokehJS ...